# MLLib Deep Dive

This walks us through though a possible solution to the [MLLib Deep Dive workshop](https://confluence.corp.ad.ctc/display/HDP/Deep-dive+Spark+MLLib).

In [1]:
%matplotlib inline

import sys, os, time, datetime
import numpy as np

# this forces the environment to spark2
# this is unnecessary if you want to use spark 1.6
sparkhome = '/usr/lib/spark2/'
os.environ['SPARK_HOME'] = sparkhome
os.environ['PYSPARK_PYTHON']="/opt/anaconda2/bin/python"
sys.path.append(sparkhome + '/python')
sys.path.append(sparkhome + '/python/lib/py4j-0.10.1-src.zip')
sys.path.append(sparkhome + '/python/lib/pyspark.zip')

import pyspark
from pyspark import SparkContext, SparkConf
from pyspark.sql import SparkSession, SQLContext, HiveContext

import pyspark.sql.functions as F

conf = SparkConf()
conf.setAppName("pyspark-lab_answerkey")
conf.set("spark.executor.memory", "20g")
conf.set("spark.executor.instances",5)
conf.set("spark.driver.memory", "20g")
conf.set("spark.executor.cores", 5)

sc = SparkContext(conf = conf)
spark = SparkSession.builder\
        .enableHiveSupport()\
        .getOrCreate()
sqlContext = spark._wrapped

In [2]:
pos = sqlContext.table('examples.mllib_lab_assignment_pos')

# clean up the massive masked store_id
# spark was having trouble merging on that bigint field
def four_digit_string(val):
    strval = str(val)
    return strval[-4:]
make_four_digit = F.udf(lambda x: four_digit_string(x))
pos = pos.withColumn('store_id', make_four_digit(pos.store_id))
pos.head()

Row(pos_transaction_id=2071964351, day_id=20150405, store_id=u'8551', product_id=1689781, deal_id=-2464554231982775937, item_actual_selling_price_amt=0.35, item_transaction_qty=6.0, item_return_quantity=0.0)

## Random forest regression

Guess the basket size based on store num, month of year and number of deals available in the store

In [3]:
# first, change the date to get the month
from pyspark.sql.types import DateType

func =  F.udf (lambda x: datetime.datetime.strptime(str(x), '%Y%M%d'), DateType())
with_dates = pos.withColumn('date', func(pos.day_id))
with_dates.head()

Row(pos_transaction_id=2071964351, day_id=20150405, store_id=u'8551', product_id=1689781, deal_id=-2464554231982775937, item_actual_selling_price_amt=0.35, item_transaction_qty=6.0, item_return_quantity=0.0, date=datetime.date(2015, 1, 5))

In [4]:
baskets = with_dates.groupby(['pos_transaction_id', 'store_id', 'date']).agg(F.sum(with_dates.item_transaction_qty))
baskets = baskets.withColumnRenamed('sum(item_transaction_qty)', 'basket_size')
baskets.head()

Row(pos_transaction_id=2071964381, store_id=u'8551', date=datetime.date(2015, 1, 5), basket_size=3.0)

In [17]:
# do a grouping to find distinct sales per store num
# this is /actually/ how many deals /sold/ that day, but is a good estimator for us.
from pyspark.sql.window import Window

store_deal_counts = pos.select('store_id', 'deal_id')\
    .distinct()\
    .groupby('store_id')\
    .count()\
    .withColumnRenamed('count', 'num_deals')
store_deal_counts = store_deal_counts.select('store_id',
                                             'num_deals',
                                             F.row_number().over(Window.partitionBy()\
                                                           .orderBy(store_deal_counts.store_id))\
                                                           .alias('store_alias'))
store_deal_counts.collect()

[Row(store_id=u'0007', num_deals=115, store_alias=1),
 Row(store_id=u'0710', num_deals=113, store_alias=2),
 Row(store_id=u'0789', num_deals=119, store_alias=3),
 Row(store_id=u'4006', num_deals=106, store_alias=4),
 Row(store_id=u'4989', num_deals=113, store_alias=5),
 Row(store_id=u'8551', num_deals=145, store_alias=6)]

In [18]:
# merge them together
joined = baskets.join(store_deal_counts, on='store_id')
joined.head()

Row(store_id=u'8551', pos_transaction_id=2071964381, date=datetime.date(2015, 1, 5), basket_size=3.0, num_deals=145, store_alias=6)

In [19]:
joined = joined.withColumn('monthnum', F.month(joined.date))
joined.head()

Row(store_id=u'8551', pos_transaction_id=2071964381, date=datetime.date(2015, 1, 5), basket_size=3.0, num_deals=145, store_alias=6, monthnum=1)

In [20]:
# make it into a label and list of features
from pyspark.mllib.regression import LabeledPoint
from pyspark.mllib.tree import RandomForest

label_and_features = joined.rdd.map(lambda x: LabeledPoint(x.basket_size, [x.store_alias, x.monthnum, x.num_deals]))

In [23]:
# the category thing is weird.
# key: the index (starting at 0) of the feature
# value: the number of categories for this feature (also indexed at 0)
model = RandomForest.trainRegressor(label_and_features, categoricalFeaturesInfo={0:7, 1:11}, numTrees=3, seed=32)

model.predict([1, 11, 110])

2.9992157148419953

## K-means clustering

Look at sales in various departments in a store. Do the stores sell different items? 

In [24]:
kmeans_data = sqlContext.table('examples.mllib_kmeans_lab_assignment_pos')
kmeans_data.head()

Row(hash8=8946748474877045509, lighting_count=651.0, snowthrower_count=4.0, golf_count=899.0, total_items=728757.0)

In [26]:
# move this into an array of features
parsed_data = kmeans_data.rdd.map(lambda x: [x.lighting_count, x.snowthrower_count, x.golf_count])
parsed_data.first()

[651.0, 4.0, 899.0]

In [30]:
from pyspark.mllib.clustering import KMeans, KMeansModel

clusters = KMeans.train(parsed_data, k=4, 
                        maxIterations=10,
                        initializationMode="random")
clusters

In [31]:
clusters.predict([651.0, 4.0, 899.0])

0

In [39]:
get_cluster = F.udf(lambda x, y, z: clusters.predict([x, y, z]))

kmeans_w_cluster = kmeans_data.withColumn('cluster_id', get_cluster(kmeans_data.lighting_count, kmeans_data.snowthrower_count, kmeans_data.golf_count))
kmeans_w_cluster.toPandas()

hash8  lighting_count  snowthrower_count  golf_count  \
0    8946748474877045509           651.0                4.0       899.0   
1   -1311085597010169475          1415.0               44.0       928.0   
2    5790637138709706180          2297.0               58.0      1648.0   
3   -7882865544843505622          1768.0               88.0      1500.0   
4    2829467192590545424           707.0               79.0       465.0   
5    1790077521091718865          2339.0               62.0      1288.0   
6    -361858321721275708          2635.0              112.0      1871.0   
7    1008613393646276787          5624.0              183.0      2811.0   
8   -7905715711766584227           926.0               66.0       619.0   
9    -965712746991792591           633.0               23.0       701.0   
10   -789835948223087947          1061.0               54.0      1883.0   
11    316417211562230037          2617.0               55.0       960.0   
12   -586365139080829717           474.0               14.0       602.0   
13   3086251142928116357          6382.0              147.0      4676.0   
14    152691144660203212          2595.0               88.0      1058.0   
15  -8434206008420196565          2073.0              122.0      2195.0   
16  -7062436394590314674          5112.0              187.0      3184.0   
17   4189623065071688358          2510.0               38.0      2649.0   
18  -9162490933138016063          2214.0              216.0      1206.0   
19   -282437733778292273           928.0               38.0      1066.0   
20   1565482275789692327          2414.0              149.0      1547.0   
21   6306487203352940340          2727.0              112.0      1307.0   
22  -2020942973217256222          1250.0               61.0      1339.0   
23   3618001307187535413          1161.0               35.0       374.0   
24   -874072590720040855          4876.0              197.0      3567.0   
25  -7786432160788417935          2574.0              140.0       702.0   
26  -1474061059118030825          2322.0               29.0       432.0   
27   -375794941778658139          1272.0               43.0       745.0   
28   4292685236291764381          2992.0               88.0      2506.0   
29  -7255121767956630249          2727.0              114.0      1400.0   
..                   ...             ...                ...         ...   
468  1029452348687620530          2755.0              150.0      1132.0   
469  3125579557950875436          2322.0              111.0      2066.0   
470  2851112438097327826          1375.0               77.0      1563.0   
471 -7292803224988165318           833.0              164.0       537.0   
472 -8511390589792299376          1887.0               44.0      1815.0   
473  4607001408098691520          2174.0               78.0      2211.0   
474 -8075649105695002681          2592.0                4.0      1179.0   
475  7141668762482669655           862.0               49.0       821.0   
476 -5231512934735745978          3652.0               38.0      1764.0   
477  1918131961983097493          2451.0               88.0      1280.0   
478  7720765621362551375          1424.0              111.0      1699.0   
479 -3487690884528144357          1395.0               32.0      1230.0   
480  -729083641170697201          3382.0              103.0      1866.0   
481   810309326758409415          2672.0               74.0      1453.0   
482   519075172284912116          2182.0              100.0       395.0   
483  9118845655044231336          1683.0              348.0       891.0   
484 -5965338941938087636           782.0               16.0       310.0   
485 -5594771028272327471          1895.0               89.0      2360.0   
486   242418397380447785          3769.0              118.0      1470.0   
487 -3761147187322448364          2030.0              129.0      1376.0   
488  7505816398455170456          3574.0              107.0      1945.0   
489  7581849591271493383          1632.0    